In [1]:
%load_ext autoreload
%autoreload 2 

In [2]:
import os
import sys

In [3]:
sys.path.append(os.path.normpath(os.path.join(os.getcwd(), '..')))

In [4]:
%env DETECTRON2_DATASETS=/runai-ivrl-scratch/students/2021-fall-sp-jellouli/semester-project/data/datasets

env: DETECTRON2_DATASETS=/runai-ivrl-scratch/students/2021-fall-sp-jellouli/semester-project/data/datasets


In [5]:
%env CUDA_VISIBLE_DEVICES=1

env: CUDA_VISIBLE_DEVICES=1


In [6]:
from detectron2.data import DatasetCatalog, MetadataCatalog, build_detection_train_loader, DatasetMapper

from pycocotools import coco

import torch

import pickle

from torch import nn

import midas_hkrm.objects.hkrm_roi_heads as hkrm
from midas_hkrm.objects.hkrm_roi_heads import ExplicitFeatureRelationshipModule, HKRMBoxHead, HKRMROIHeads

In [7]:
import detectron2
from detectron2.utils.logger import setup_logger
setup_logger()
from detectron2 import model_zoo
from detectron2.engine import DefaultPredictor
from detectron2.config import get_cfg
from detectron2.utils.visualizer import Visualizer
from detectron2.data import MetadataCatalog, DatasetCatalog
from detectron2.modeling import build_model
from detectron2.config.config import CfgNode

from midas_hkrm.utils.objects_utils import construct_config, get_hkrm_model, HKRMTrainer

from detectron2.engine.defaults import DefaultTrainer

In [8]:
cfg = construct_config()

In [9]:
cfg.SOLVER.IMS_PER_BATCH = 10
cfg.MODEL.RPN.POST_NMS_TOPK_TRAIN = 128 # like the paper
# cfg.OUTPUT_DIR = 'OUTPUT_DIR'
cfg.TEST.EVAL_PERIOD = 1

In [10]:
trainer = HKRMTrainer(cfg)

[11/23 19:11:56 d2.engine.defaults]: Model:
GeneralizedRCNN(
  (backbone): FPN(
    (fpn_lateral2): Conv2d(256, 256, kernel_size=(1, 1), stride=(1, 1))
    (fpn_output2): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (fpn_lateral3): Conv2d(512, 256, kernel_size=(1, 1), stride=(1, 1))
    (fpn_output3): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (fpn_lateral4): Conv2d(1024, 256, kernel_size=(1, 1), stride=(1, 1))
    (fpn_output4): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (fpn_lateral5): Conv2d(2048, 256, kernel_size=(1, 1), stride=(1, 1))
    (fpn_output5): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (top_block): LastLevelMaxPool()
    (bottom_up): ResNet(
      (stem): BasicStem(
        (conv1): Conv2d(
          3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False
          (norm): FrozenBatchNorm2d(num_features=64, eps=1e-05)
        )
      )
      (res

[11/23 19:12:12 d2.data.datasets.coco]: Loading /runai-ivrl-scratch/students/2021-fall-sp-jellouli/semester-project/data/datasets/coco/annotations/instances_train2017.json takes 15.62 seconds.
[11/23 19:12:13 d2.data.datasets.coco]: Loaded 118287 images in COCO format from /runai-ivrl-scratch/students/2021-fall-sp-jellouli/semester-project/data/datasets/coco/annotations/instances_train2017.json
[11/23 19:12:20 d2.data.build]: Removed 1021 images with no usable annotations. 117266 images left.
[11/23 19:12:27 d2.data.build]: Distribution of instances among all 80 categories:
|   category    | #instances   |   category   | #instances   |   category    | #instances   |
|:-------------:|:-------------|:------------:|:-------------|:-------------:|:-------------|
|    person     | 257253       |   bicycle    | 7056         |      car      | 43533        |
|  motorcycle   | 8654         |   airplane   | 5129         |      bus      | 6061         |
|     train     | 4570         |    truck  

In [11]:
for m in trainer.model.backbone.bottom_up.parameters():
    m.requires_grad = False

In [12]:
trainer.resume_or_load()

[11/23 19:12:33 d2.checkpoint.c2_model_loading]: Renaming Caffe2 weights ......
[11/23 19:12:33 d2.checkpoint.c2_model_loading]: Following weights matched with submodule backbone.bottom_up:
| Names in Model    | Names in Checkpoint       | Shapes                                          |
|:------------------|:--------------------------|:------------------------------------------------|
| res2.0.conv1.*    | res2_0_branch2a_{bn_*,w}  | (64,) (64,) (64,) (64,) (64,64,1,1)             |
| res2.0.conv2.*    | res2_0_branch2b_{bn_*,w}  | (64,) (64,) (64,) (64,) (64,64,3,3)             |
| res2.0.conv3.*    | res2_0_branch2c_{bn_*,w}  | (256,) (256,) (256,) (256,) (256,64,1,1)        |
| res2.0.shortcut.* | res2_0_branch1_{bn_*,w}   | (256,) (256,) (256,) (256,) (256,64,1,1)        |
| res2.1.conv1.*    | res2_1_branch2a_{bn_*,w}  | (64,) (64,) (64,) (64,) (64,256,1,1)            |
| res2.1.conv2.*    | res2_1_branch2b_{bn_*,w}  | (64,) (64,) (64,) (64,) (64,64,3,3)             |
| res2.1.c

Some model parameters or buffers are not found in the checkpoint:
backbone.fpn_lateral2.{bias, weight}
backbone.fpn_lateral3.{bias, weight}
backbone.fpn_lateral4.{bias, weight}
backbone.fpn_lateral5.{bias, weight}
backbone.fpn_output2.{bias, weight}
backbone.fpn_output3.{bias, weight}
backbone.fpn_output4.{bias, weight}
backbone.fpn_output5.{bias, weight}
proposal_generator.rpn_head.anchor_deltas.{bias, weight}
proposal_generator.rpn_head.conv.{bias, weight}
proposal_generator.rpn_head.objectness_logits.{bias, weight}
roi_heads.box_head.attribute_feature_transform.feature_transform.{bias, weight}
roi_heads.box_head.attribute_feature_transform.relationship_scorer.fc_1.0.{bias, weight}
roi_heads.box_head.attribute_feature_transform.relationship_scorer.fc_2.0.{bias, weight}
roi_heads.box_head.attribute_feature_transform.relationship_scorer.fc_3.0.{bias, weight}
roi_heads.box_head.attribute_feature_transform.relationship_scorer.fc_output.{bias, weight}
roi_heads.box_head.base_box_head.fc1.

In [13]:
cfg.MODEL.DEVICE

'cuda'

In [14]:
trainer.train()

[11/23 19:12:33 d2.engine.train_loop]: Starting training from iteration 0
[11/23 19:12:36 d2.data.datasets.coco]: Loaded 5000 images in COCO format from /runai-ivrl-scratch/students/2021-fall-sp-jellouli/semester-project/data/datasets/coco/annotations/instances_val2017.json
[11/23 19:12:37 d2.data.build]: Distribution of instances among all 80 categories:
|   category    | #instances   |   category   | #instances   |   category    | #instances   |
|:-------------:|:-------------|:------------:|:-------------|:-------------:|:-------------|
|    person     | 10777        |   bicycle    | 314          |      car      | 1918         |
|  motorcycle   | 367          |   airplane   | 143          |      bus      | 283          |
|     train     | 190          |    truck     | 414          |     boat      | 424          |
| traffic light | 634          | fire hydrant | 101          |   stop sign   | 75           |
| parking meter | 60           |    bench     | 411          |     bird      |

ValueError: not enough values to unpack (expected 2, got 1)